# Intwari ATLAS v2 - Features

## Setup & Data Ingestion

In [ ]:
# the great wall of imports
import ee, os, math
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from featuretoolkit import src as ftk
import geemap as gmp

In [ ]:
# load env file and take in settings from config
from dotenv import load_dotenv
import yaml

load_dotenv()
ee.Authenticate()
ee.Initialize(project=os.getenv('projectkey'))

with open('config.yaml','r') as f:
    config=yaml.safe_load(f)
    
SEED=config['main']['seed'] # 42
np.random.seed(42)
tf.random.set_seed(42)

In [18]:
gfld=pd.read_excel('database/19_10 Landslide database master.xlsx')
# courtesy of Professor Dave Petley, Vice-Chancellor of UHull

coolr=pd.read_csv('database/nasa_global_landslide_catalog_point.csv')
# courtesy of NASA Global Landslide Catalog @ https://maps.nccs.nasa.gov/arcgis/apps/MapAndAppGallery/index.html?appid=574f26408683485799d02e857e5d9521 (landslide reports points csv)

In [ ]:
# all images used in sampling
nasa_dem=ee.Image("USGS/SRTMGL1_003").resample('bilinear')
openlandmap_sand=ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02")
openlandmap_clay=ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02")
esa_worldcover=ee.ImageCollection("ESA/WorldCover/v200").first()
isric_soilgrids=ee.ImageCollection("ISRIC/SoilGrids250m/v2_0")
nasa_gpm=ee.ImageCollection("NASA/GPM_L3/IMERG_V07")
copernicus_era5=ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
nasa_modis_terraveg=ee.ImageCollection("MODIS/061/MOD13Q1")

In [ ]:
testpointer=ee.Geometry.Point(30.060611,-1.944306) # western kigali
testtimestamp=1678838400000 # mar 15 2023

## Exploratory Data Analysis & Pre-preprocessing

In [ ]:
gfld.head()

,LandslideN,Country,Date,Day,Month,Year,Continent,Fatalities,Trigger,Location_M,...,Longitude,Location_R,Report 1,Source 1,Report 2,Source 2,Report 3,Source 3,Report 4,Source 4
0,1,Mozambique,2004-01-02,2,1,2004,E. Africa,6,unknown,Sofala,...,34.675341,Tsiquiri region Sofala province,NaN,http://allafrica.com/stories/200401050521.html,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Pakistan,2004-01-02,2,1,2004,S. Asia,3,construction,Shogran,...,73.462259,Shogran,NaN,http://paktribune.com/news/index.php?id=50368,NaN,NaN,NaN,NaN,NaN,NaN
2,3,China,2004-01-03,3,1,2004,E. Asia,16,unknown,Linfen,...,111.372850,Jingpo Village Linfen City Shanxi N. China,NaN,http://news.xinhuanet.com/english/2004-01/04/c...,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Indonesia,2004-01-04,4,1,2004,S.E. Asia,4,rainfall,Wanahayu,...,108.303381,Wanahayu village Maja subdistrict Majalengka...,NaN,http://www.antara.co.id/e_berita.asp?id=129215...,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Mexico,2004-01-11,11,1,2004,C. America,2,rainfall,Pichucalco,...,-93.188786,Pichucalco c.80 kms north of Tuxtla Gutierrez...,NaN,http://www.contracostatimes.com/mld/cctimes/ne...,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
coolr.head()

,OID,source_name,source_link,event_id,event_date,event_time,event_title,event_description,location_description,location_accuracy,...,event_import_id,latitude,longitude,country_name,country_code,admin_division_name,gazetteer_closest_point,gazetteer_distance,submitted_date,last_edited_date
0,-1,NASA SERVIR Science Coordination Office & SERV...,https://servir.adpc.net/,11757,NaN,NaN,Landslide in Myanmar,NaN,NaN,exact,...,178,18.741125,94.693509,Myanmar,MM,Rakhine,Buyo Chaung,18.35,8/17/2018 0:00:00,NaN
1,-1,The Bolton News,http://www.theboltonnews.co.uk/news/14174289.R...,9148,12/30/2015 0:00:00,NaN,Forest Road,FAMILIES were forced from their homes after a ...,Forest Road,exact,...,9148,53.596700,-2.460000,United Kingdom,GB,England,Bolton,1.91,7/25/2016 17:42:30,NaN
2,-1,The Arunachal Times,http://www.arunachaltimes.in/six-killed-in-meg...,10987,6/17/2017 0:00:00,NaN,Fatal landslide in Umiam,Landslide following continuous rain kills 3 in...,"Umiam, Ribhoi, Meghalaya",5km,...,NaN,25.678368,91.923434,India,IN,Meghalaya,Shillong,6.30,7/10/2017 18:45:55,NaN
3,-1,newvision,http://www.newvision.co.ug/D/8/12/719718,1862,5/15/2010 0:00:00,NaN,"road to Kanungu district at Kamujegye(?), Rwer...","In Rukungiri district, motor traffic stalled a...","road to Kanungu district at Kamujegye(?), Rwer...",10km,...,1862,-0.788800,29.821400,Uganda,UG,Kanungu,Kinyasano,10.54,4/1/2014 0:00:00,NaN
4,-1,mcot,http://www.mcot.net/cfcustom/cache_page/187154...,3305,3/27/2011 0:00:00,NaN,Surat Thani province,"CHUMPHON, March 27 -- Several tourists and vil...",Surat Thani province,unknown,...,3305,9.010300,99.088600,Thailand,TH,Surat Thani,Khiri Rat Nikhom,13.55,4/1/2014 0:00:00,NaN


In [ ]:
coolr=ftk.process_coolr(coolr) # filters & cleans coolr dataset (ie. dropping duplicates, dropping non-meteorologically triggered landslides, converting dates to timestamps, etc.)
coolr

,time_start,time_end,spatial_uncertainty,latitude_center,longitude_center
0,1.497658e+12,1.497744e+12,5000.0,25.678368,91.923434
1,1.273882e+12,1.273968e+12,10000.0,-0.788800,29.821400
2,1.271722e+12,1.271808e+12,1000.0,26.196500,91.763000
3,1.136333e+12,1.136419e+12,5000.0,-6.934300,110.196900
4,1.440720e+12,1.440806e+12,25000.0,23.449500,92.811200
...,...,...,...,...,...
6817,1.580947e+12,1.581034e+12,500.0,37.857066,-81.014200
6818,1.676551e+12,1.676551e+12,500.0,38.388959,-81.658976
6819,1.502410e+12,1.502496e+12,NaN,38.711048,-77.083595
6820,1.739578e+12,1.739664e+12,500.0,38.395343,-82.300335


In [ ]:
gfld=ftk.process_gfld(gfld) # filters & cleans gfld dataset (ie. dropping duplicates, dropping non-meteorologically triggered landslides, converting dates to timestamps, etc.)
gfld

,time_start,time_end,spatial_uncertainty,latitude_center,longitude_center
0,1.073174e+12,1.073261e+12,1271.402335,-6.925276,108.303381
1,1.073779e+12,1.073866e+12,14794.854925,17.552324,-93.188786
2,1.073779e+12,1.073866e+12,10062.257299,7.092358,80.886529
3,1.074038e+12,1.074125e+12,17313.149141,1.382049,110.117929
4,1.074125e+12,1.074211e+12,9063.467536,-22.585616,-43.327366
...,...,...,...,...,...
3596,1.513382e+12,1.513469e+12,4971.103785,11.560030,124.552919
3597,1.513382e+12,1.513469e+12,1164.719591,11.623736,124.426184
3598,1.513382e+12,1.513469e+12,1120.598696,-43.384466,-72.407090
3599,1.513901e+12,1.513987e+12,2336.392335,7.886669,125.170705


In [ ]:
positives,dropped=ftk.concatenate(coolr,gfld) # smart concatenation function; removes inter-dataset duplicate events, adds regions & folds, sets ids
positives

,event_id,time_start,time_end,spatial_uncertainty,latitude_center,longitude_center,region_id,fold_id
0,0,1.497658e+12,1.497744e+12,5.000000,25.678368,91.923434,823ce7fffffffff,813cfffffffffff
1,1,1.273882e+12,1.273968e+12,10.000000,-0.788800,29.821400,826adffffffffff,816afffffffffff
2,2,1.271722e+12,1.271808e+12,1.000000,26.196500,91.763000,823ce7fffffffff,813cfffffffffff
3,3,1.136333e+12,1.136419e+12,5.000000,-6.934300,110.196900,828d8ffffffffff,818dbffffffffff
4,4,1.440720e+12,1.440806e+12,25.000000,23.449500,92.811200,823cc7fffffffff,813cfffffffffff
...,...,...,...,...,...,...,...,...
8696,8696,1.513382e+12,1.513469e+12,4.971104,11.560030,124.552919,826867fffffffff,81687ffffffffff
8697,8697,1.513382e+12,1.513469e+12,1.164720,11.623736,124.426184,826867fffffffff,81687ffffffffff
8698,8698,1.513382e+12,1.513469e+12,1.120599,-43.384466,-72.407090,81cebffffffffff-OTHER,81cebffffffffff
8699,8699,1.513901e+12,1.513987e+12,2.336392,7.886669,125.170705,82685ffffffffff,81687ffffffffff


In [ ]:
dropped # dropped points (interdataset duplicates)

,coolr_id,gfld_id,date,distance_m,coolr_rad_m,gfld_rad_m,time_start_coolr,time_end_coolr,time_start_gfld,time_end_gfld
0,0,3429,2017-06-17 00:00:00+00:00,34953.441692,5000.0,30182.649115,2017-06-17 00:00:00+00:00,2017-06-18 00:00:00+00:00,2017-06-17 00:00:00+00:00,2017-06-18 00:00:00+00:00
1,6,1720,2010-08-07 00:00:00+00:00,51200.584803,50000.0,125000.011648,2010-08-06 00:00:00+00:00,2010-08-07 00:00:00+00:00,2010-08-07 00:00:00+00:00,2010-08-08 00:00:00+00:00
2,13,2965,2015-07-11 00:00:00+00:00,9809.581029,5000.0,6077.195929,2015-07-11 00:00:00+00:00,2015-07-12 00:00:00+00:00,2015-07-11 00:00:00+00:00,2015-07-12 00:00:00+00:00
3,16,2525,2013-07-22 00:00:00+00:00,6238.742183,5000.0,2658.011264,2013-07-22 00:00:00+00:00,2013-07-23 00:00:00+00:00,2013-07-22 00:00:00+00:00,2013-07-23 00:00:00+00:00
4,18,1642,2010-06-17 00:00:00+00:00,15423.308546,10000.0,18805.863846,2010-06-16 00:00:00+00:00,2010-06-17 00:00:00+00:00,2010-06-17 00:00:00+00:00,2010-06-18 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...
807,6764,2822,2014-08-27 00:00:00+00:00,200.898442,5000.0,344.730880,2014-08-27 00:00:00+00:00,2014-08-28 00:00:00+00:00,2014-08-27 00:00:00+00:00,2014-08-28 00:00:00+00:00
808,6773,2820,2014-08-27 00:00:00+00:00,21507.213459,1000.0,23164.771914,2014-08-27 00:00:00+00:00,2014-08-28 00:00:00+00:00,2014-08-27 00:00:00+00:00,2014-08-28 00:00:00+00:00
809,6780,2508,2013-07-10 00:00:00+00:00,72.035604,5000.0,208.390930,2013-07-10 00:00:00+00:00,2013-07-11 00:00:00+00:00,2013-07-10 00:00:00+00:00,2013-07-11 00:00:00+00:00
810,6783,1761,2010-08-28 00:00:00+00:00,1093.202153,5000.0,1937.731192,2010-08-27 00:00:00+00:00,2010-08-28 00:00:00+00:00,2010-08-28 00:00:00+00:00,2010-08-29 00:00:00+00:00


In [ ]:
# here's the full dataset of positives which will be processed itself and also used to determine the background sampling distribution
map=gmp.Map()
map.addLayer(ee.FeatureCollection([ee.Geometry.Point(lon,lat) for lon,lat in zip(positives['longitude_center'],positives['latitude_center'])]).style(**{'color':'blue','pointSize':1}),{},'Landslide Events')
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

## Positive Sampling (The Phi Function)

In [28]:
# get stuff from config for later
CRS_METRIC=config['sampling']['crs']
SCALE_METRIC=config['sampling']['scale']
BUFFER=config['sampling']['t_buffer'] # adds a small temporal buffer to partly avoid filtering erros with dynamic datasets
soil_levels=list(config['features']['soil_moist_mean_layers'].keys())
soil_depths=list(config['features']['soil_moist_mean_layers'].values())

# import all necessary images/imagecollections
nasa_dem=ee.Image("USGS/SRTMGL1_003").resample('bilinear')
openlandmap_sand=ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").resample('bilinear')
openlandmap_clay=ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").resample('bilinear')
esa_worldcover=ee.ImageCollection("ESA/WorldCover/v200").first().resample('bilinear').rename('land_class')
nasa_gpm=ee.ImageCollection("NASA/GPM_L3/IMERG_V07")
copernicus_era5=ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select([*[f'volumetric_soil_water_layer_{level}' for level in soil_levels],'potential_evaporation_hourly'])
nasa_modis_terraveg=ee.ImageCollection("MODIS/061/MOD13Q1")
upa=ee.Image('MERIT/Hydro/v1_0_1').resample('bilinear').select('upa') # upstream drainage area to calculate sca

# the phi function: maps a pointer & timestamp to a set of geospatial features in that region in spacetime (precipitation, slope, etc.) using well-optimized google earth engine api calls
# one major efficiency gain made here is splitting the features into static & dynamic branches which are processed separately which means static features can be sampled once without regard for time
def phi(pointer:ee.geometry.Geometry,radius:float,radius_max:float,time_start:float,time_end:float,time_max:float,seed=SEED):

    # figure out K (number of subsamples; see explanation here: https://www.desmos.com/calculator/dem1lu0xt5)
    K_MAX=config['sampling']['K_max_events']
    T_MULTIPLIER=config['sampling']['t_multiplier']
    def scaleK(radius,interval): return round((interval*T_MULTIPLIER*(K_MAX-1))/(time_max*radius_max**2)*radius**2+1)
    K=scaleK(radius=radius,interval=abs(time_end-time_start))

    # helper function to convert pointer & radius (km) to a roughly circular ee geometry object for subsampling
    def uncertainty_region(pointer:ee.geometry.Geometry=pointer,radius:float=radius): 
        return pointer.buffer(radius*1000)
    region=uncertainty_region()

    # helper function to randomly & uniformly samples the spacetime area specified by parameters K times, outputs only specific timestamps & coordinate locations
    def generate_samples(region:ee.geometry.Geometry=region,time_start:float=time_start,time_end:float=time_end,K:int=K):
        if time_start == None and time_end == None: return ee.FeatureCollection.randomPoints(region=region,points=K,seed=seed)
        start=ee.Date(time_start)
        duration=ee.Date(time_end).difference(start,'second').multiply(1000)
        pts=ee.FeatureCollection.randomPoints(region=region,points=K,seed=seed).randomColumn('u',seed=seed+1)
        def _add_time(f):
            ts=start.millis().add(duration.multiply(f.get('u')))
            return f.set({'ts':ts,'date':ee.Date(ts)})
        return pts.map(_add_time)
    global subsamples
    subsamples=generate_samples()

    # helper function that figures out the number of pixels in a given image that overlap with a given geometry object 
    def overlap_count(image:ee.Image,region:ee.geometry.Geometry=region):
        proj=image.projection()

        # count unmasked pixels in rasterized mask, return all pixel overlaps
        counts=ee.Image.constant(1).rename('m').clip(region).reproject(proj).reduceRegion(reducer=ee.Reducer.count(),geometry=region,scale=proj.nominalScale(),tileScale=2)
        return ee.Number(counts.get('m')).getInfo()
    
    # child phi function that specializes in dynamic imagecollections (in this case those with a <1d temporal cadence)
    def phi_dynamic(subsamples:ee.FeatureCollection=subsamples,region:ee.geometry.Geometry=region,time_start:float=time_start,time_end:float=time_end):
        
        # define gpm imagecollection and select only precipitation
        gpm=nasa_gpm.select('precipitation')

        # define era5 imagecollections, flatten temporally
        era5_pev=copernicus_era5.select('potential_evaporation_hourly') # potential evaporation
        era5_vsw=copernicus_era5.select(['volumetric_soil_water_layer_1','volumetric_soil_water_layer_4']) # volumetric soil water content

        # helper function to convert ic hourly cadence to daily aggregate
        def hourly_to_daily_mean(ic:ee.ImageCollection):
            # get start & end dates of ic
            start_date=ee.Date(ic.first().get('system:time_start'))
            end_date=ee.Date(ic.sort('system:time_start',False).first().get('system:time_start'))
            # helper function to get mean for a single day
            def _day_mean(offset):
                current_day=start_date.advance(offset,'day')
                next_day=current_day.advance(1,'day')
                # filter collection for current day, take mean for all images in filtered ic, set time start property for new resultant img & return
                mean_img=ic.filterDate(current_day,next_day).mean()
                return mean_img.set('system:time_start',current_day.millis())
            # derive number of days, convert list of images into ic after mapping over generated list of daily timestamps
            num_days=end_date.difference(start_date,'day').round()
            return ee.ImageCollection.fromImages(ee.List.sequence(0,num_days.subtract(1)).map(_day_mean))

        # if the overlap count is 1 for both datasets then, necessarily and logically, every subsample falls within the same pixel. therefore, no need to subsample more than once for any given temporal step
        if overlap_count(nasa_gpm.first())==1 and overlap_count(copernicus_era5.first())==1:
            centroid=region.centroid() # get region centroid for point sampling

            def sample_uniform(pointer:ee.geometry.Geometry,ts:float=time_start):
                # initialize empty dict for storing names & properties as key-val pairs to concatenate to the existing subsampling fc
                new_properties=dict()

                # derive hourly precipitation sums over past 1, 6, 24, 72, 240, 720 hrs
                def precip_sum(timestamp:float,hrs:int,pointer:ee.geometry.Geometry=pointer):
                    return gpm.filterDate(int(timestamp-(hrs*3600000+BUFFER)),int(timestamp)).sum().sample(pointer).getInfo()['features'][0]['properties']['precipitation']
                for hrs in config['features']['windows']['precip_sum_h']:new_properties[f'precip_mm_{hrs}h_sum']=precip_sum(int(ts),hrs)

                # derive hourly precipitation maxes over past 6, 12, 24 hrs
                def precip_max(timestamp:float,hrs:int,pointer:ee.geometry.Geometry=pointer):
                    return gpm.filterDate(int(timestamp-(hrs*3600000+BUFFER)),int(timestamp)).max().sample(pointer).getInfo()['features'][0]['properties']['precipitation']
                for hrs in config['features']['windows']['precip_max_h']:new_properties[f'precip_mm_{hrs}h_max']=precip_max(int(ts),hrs)

                # derive the number of half hours for 72 hours before timestamp where precipitation exceeded 5mm (hits)
                def precip_half_hours_5mm(timestamp:float,hrs:int,pointer:ee.geometry.Geometry=pointer):
                    return gpm.filterDate(int(timestamp-(4*3600000+BUFFER)),int(timestamp)).map(lambda img: img.multiply(0.5).gt(10).rename('hits').toUint8().updateMask(img.mask())).sum().sample(pointer).getInfo()['features'][0]['properties']['hits']
                for hrs in config['features']['windows']['precip_hits_h']:new_properties[f'precip_hits_{hrs}h']=precip_half_hours_5mm(int(ts),hrs)

                # derive mean hourly volumetric soil water content (soil moisture) over past 24, 72 hours @ depth levels 1 (0-7cm), 4 (100-289cm); (4 bands total)
                def soil_moist_mean(timestamp:float,hrs:int,pointer:ee.geometry.Geometry=pointer):
                    return era5_vsw.filterDate(timestamp-(hrs*3600000+BUFFER),int(timestamp)).mean().select([f'volumetric_soil_water_layer_{level}' for level in soil_levels],[f'vsw_{depth}_{hrs}h' for depth in soil_depths]).sample(pointer).getInfo()['features'][0]['properties']
                for hrs in config['features']['windows']['soil_moist_mean_h']:new_properties.update(soil_moist_mean(int(ts),hrs))

                # derive soil moisture anomaly over 90 days (z-score) for just depth level 1 (0-7cm)
                def soil_moist_anom(timestamp:float,days:int,pointer:ee.geometry.Geometry=pointer):
                    # get daily soil moisture data for specified time range
                    daily_collection=hourly_to_daily_mean(era5_vsw.filterDate(int(timestamp-days*86400000),int(timestamp)).select('volumetric_soil_water_layer_1'))
                    # reduce (reducer computes mean & stdev in a single pass for efficiency) entire image collection to get mean and standard deviation
                    stats_image=daily_collection.reduce(ee.Reducer.mean().combine(ee.Reducer.stdDev(),None,True))
                    mean_image=stats_image.select('volumetric_soil_water_layer_1_mean')
                    stdev_image=stats_image.select('volumetric_soil_water_layer_1_stdDev').max(1e-6)
                    # calculate final z-score per pixel, conduct final sampling & return
                    return daily_collection.sort('system:time_start',False).first().subtract(mean_image).divide(stdev_image).rename(f'soil_moist_anom_{days}d').sample(pointer).getInfo()['features'][0]['properties']
                for days in config['features']['windows']['soil_moist_anom_d']:new_properties.update(soil_moist_anom(int(ts),days))

                # derive slope of linearly fitted trend line for soil moisture over antecedent 7d window for just depth level 1 (0-7cm)
                def soil_moist_trend(timestamp:float,days:int,pointer:ee.geometry.Geometry=pointer):
                    daily_collection=hourly_to_daily_mean(era5_vsw.filterDate(int(timestamp-days*86400000),int(timestamp)).select('volumetric_soil_water_layer_1'))
                    # add time-in-days since start as band 't'
                    def _add_t(img):return (img.rename('sm').addBands(ee.Image.constant(ee.Date(img.get('system:time_start')).difference(ee.Date(timestamp).advance(-days,'day'),'day')).rename('t')).toFloat())
                    # linear fit; y=scale*x+offset (where x='t' (in days) & y='moisture')
                    fit=daily_collection.map(_add_t).select(['t','sm']).reduce(ee.Reducer.linearFit())
                    return fit.select('scale').rename(f'soil_moist_trend_{days}d').sample(pointer).getInfo()['features'][0]['properties']
                for hrs in config['features']['windows']['soil_moist_trend_d']:new_properties.update(soil_moist_trend(int(ts),hrs))

                # derive sum of potential evaporation over antecedent 7d window
                def pev_sum(timestamp:float,days:int,pointer:ee.geometry.Geometry=pointer):
                    # potential evaporation sum for days before timestamp
                    return era5_pev.filterDate(int(timestamp)-(days*86400000+BUFFER),int(timestamp)).select('potential_evaporation_hourly').sum().rename(f'pev_sum_{days}d').sample(pointer).getInfo()['features'][0]['properties']
                pev=pev_sum(int(ts),config['features']['windows']['pev_sum_d'])
                new_properties.update(pev)

                # derive moisture deficit calculated by the ReLU of potential evaporation minus precipitation over antecedent 7d window
                new_properties['moisture_deficit']=max(pev.values[0]-precip_sum(int(ts),config['features']['windows']['pev_sum_d']*24),0)

                # return dict
                return new_properties

            if time_start==time_end: # use single spatiotemporal sampling if time interval is also singular
                # retrieve all uniform dynamic info & add to subsamples
                def _add_property_uniform(feature): return feature.set(sample_uniform(centroid)) # adds channels (as properties) to fc via mapping
                subsamples=subsamples.map(_add_property_uniform) # update fc
            else:
                # figure out which buckets to sample (break up the day into 24 1hr long buckets so channels can be sampled bucketwise rather than samplewise -- some buckets may be empty so only calculate populated ones)
                occupied_buckets=list(np.unique([math.floor(24*subsample) for subsample in subsamples.aggregate_array('u').getInfo()]))
                timestamps=[time_start+bucket*86400000+450000 for bucket in occupied_buckets] # calculate exact quarter-way timestamp for each occupied bucket
                buckets=dict() # empty dict for bucket channel values after sampling; will be populated with keys as bucket hour & vals as outputs of sample_uniform() (dicts)

                # populate dict bucketwise using uniform sampling at each bucket
                for bkt,tss in zip(occupied_buckets,timestamps):buckets.update({int(bkt):sample_uniform(centroid,int(tss))})

                # add properties to each feature based on bucket for mapping, map to subsamples fc
                def _add_property_temp(feature): 
                    bucket=math.floor(24*feature.get('u'))
                    return feature.set(buckets[bucket])
                subsamples=subsamples.map(_add_property_temp)
        else:
            occupied_buckets=list(np.unique([math.floor(24*subsample) for subsample in subsamples.aggregate_array('u').getInfo()]))
            timestamps=[time_start+bucket*86400000+450000 for bucket in occupied_buckets] # calculate exact quarter-way timestamp for each occupied bucket

            # derive hourly precipitation sums over past 1, 6, 24, 72, 240, 720 hrs 
            def precip_sum_snapshot(timestamp:float,hrs:int):
                # sum precipitation for hours before timestamp
                return gpm.filterDate(int(timestamp-(hrs*3600000+BUFFER)),int(timestamp)).sum().rename(f'precip_mm_{hrs}h_sum')
            precip_sum_imgs=[ee.Image.cat([precip_sum_snapshot(int(ts),hrs) for ts in timestamps]) for hrs in config['features']['windows']['precip_sum_h']]

            # derive hourly precipitation maxes over past 6, 12, 24 hrs
            def precip_max_snapshot(timestamp:float,hrs:int):
                # max precipitation for hours before timestamp
                return gpm.filterDate(int(timestamp-(hrs*3600000+BUFFER)),int(timestamp)).max().rename(f'precip_mm_{hrs}h_max')
            precip_max_imgs=[ee.Image.cat([precip_max_snapshot(int(ts),hrs) for ts in timestamps]) for hrs in config['features']['windows']['precip_max_h']]

            # derive the number of half hours for 72 hours before timestamp where precipitation exceeded 5mm (hits)
            def precip_half_hours_5mm_snapshot(timestamp:float,hrs:int):
                # the number of half_hours for hours before timestamp where precipitation exceeded 5mm
                return gpm.filterDate(int(timestamp-(hrs*3600000+BUFFER)),int(timestamp)).map(lambda img: img.multiply(0.5).gt(10).rename('hit').toUint8().updateMask(img.mask())).sum().rename(f'precip_half_hours_5mm_{hrs}h')
            precip_hits_imgs=[ee.Image.cat([precip_half_hours_5mm_snapshot(int(ts),hrs) for ts in timestamps]) for hrs in config['features']['windows']['precip_hits_h']]

            # derive mean hourly volumetric soil water content (soil moisture) over past 24, 72 hours @ depth levels 1 (0-7cm), 4 (100-289cm); (4 bands total)
            def soil_moist_mean_snapshot(timestamp:float,hrs:int): 
                return era5_vsw.filterDate(int(timestamp-(hrs*3600000+BUFFER)),int(timestamp)).mean().select([f'volumetric_soil_water_layer_{level}' for level in soil_levels],[f'vsw_{depth}_{hrs}h' for depth in soil_depths])
            soil_moist_mean_imgs=[[ee.Image.cat([soil_moist_mean_snapshot(int(ts),hrs).select(f'vsw_{depth}_{hrs}h') for ts in timestamps]) for hrs in config['features']['windows']['soil_moist_mean_h']] for depth in soil_depths]
            soil_moist_mean_imgs_flat=[img for imglist in soil_moist_mean_imgs for img in imglist]

            # derive soil moisture anomaly over 90 days (z-score) for just depth level 1 (0-7cm)
            def soil_moist_anom_snapshot(timestamp:float,days:int):
                # get daily soil moisture data for specified time range
                daily_collection=hourly_to_daily_mean(era5_vsw.filterDate(int(timestamp-days*86400000),int(timestamp)).select('volumetric_soil_water_layer_1'))
                # reduce (reducer computes mean & stdev in a single pass for efficiency) entire image collection to get mean and standard deviation
                stats_image=daily_collection.reduce(ee.Reducer.mean().combine(ee.Reducer.stdDev(),None,True))
                mean_image=stats_image.select('volumetric_soil_water_layer_1_mean')
                stdev_image=stats_image.select('volumetric_soil_water_layer_1_stdDev').max(1e-6)
                # calculate final z-score per pixel, conduct final sampling & return
                return daily_collection.sort('system:time_start',False).first().subtract(mean_image).divide(stdev_image).rename(f'soil_moist_anom_{days}d')
            soil_moist_anom_imgs=[ee.Image.cat([soil_moist_anom_snapshot(int(ts),days) for ts in timestamps]) for days in config['features']['windows']['soil_moist_anom_d']]

            # derive slope of linearly fitted trend line for soil moisture over antecedent 7d window for just depth level 1 (0-7cm)
            def soil_moist_trend_snapshot(timestamp:float,days:int):
                daily_collection=hourly_to_daily_mean(era5_vsw.filterDate(int(timestamp-days*86400000),int(timestamp)).select('volumetric_soil_water_layer_1'))
                # add time-in-days since start as band 't'
                def _add_t(img):return (img.rename('sm').addBands(ee.Image.constant(ee.Date(img.get('system:time_start')).difference(ee.Date(timestamp).advance(-days,'day'),'day')).rename('t')).toFloat())
                # linear fit; y=scale*x+offset (where x='t' (in days) & y='moisture')
                fit=daily_collection.map(_add_t).select(['t','sm']).reduce(ee.Reducer.linearFit())
                return fit.select('scale').rename(f'soil_moist_trend_{days}d')
            soil_moist_trend_imgs=[ee.Image.cat([soil_moist_trend_snapshot(int(ts),days) for ts in timestamps]) for days in config['features']['windows']['soil_moist_trend_d']]

            # derive sum of potential evaporation over antecedent 7d window
            def pev_sum_snapshot(timestamp:float,days:int):
                # potential evaporation sum for days before timestamp
                return era5_pev.filterDate(int(timestamp-(days*86400000+BUFFER)),int(timestamp)).select('potential_evaporation_hourly').sum().rename(f'pev_sum_{days}d')
            pev_sum_list=[pev_sum_snapshot(int(ts),config['features']['windows']['pev_sum_d']) for ts in timestamps]
            pev_sum_imgs=[ee.Image.cat(pev_sum_list)]

            # derive moisture deficit calculated by the ReLU of potential evaporation minus precipitation over antecedent 7d window
            def moisture_deficit_snapshot():
                return ee.Image.cat([pev.subtract(precip).rename(f'moisture_deficit_7d') for pev,precip,idx in zip (pev_sum_list,[precip_sum_snapshot(int(ts),config['features']['windows']['pev_sum_d']*24) for ts in timestamps],occupied_buckets)])
            moisture_deficit_imgs=[moisture_deficit_snapshot()]
        
            # unwrap & compile all images, run sampling for each image after selecting relevant bucket then add to dict featurewise
            all_imgs=[*precip_sum_imgs,*precip_max_imgs,*precip_hits_imgs,*soil_moist_mean_imgs_flat,*soil_moist_anom_imgs,*soil_moist_trend_imgs,*pev_sum_imgs,*moisture_deficit_imgs]

            # need to express the list of occupied buckets as ee numbers for selection in the next step
            occupied_buckets_objs=[ee.Number(int(bucket)) for bucket in occupied_buckets]
            def _sample_channels_bkt(feature): # map bucketed samples to features
                bucket=ee.Number(feature.get('u')).multiply(24).floor().int() # need to do all these operations serverside since its a mapping fn
                img=ee.Image.cat([img.select(ee.List(occupied_buckets_objs).indexOf(bucket)) for img in all_imgs]) # making sure only the relevant band (temporal bucket) is selected then restacking bands
                sample_fc=img.sample(feature.geometry()).first().toDictionary()
                return feature.set(sample_fc)
                    
            # update subsamples fc by mapping bucketed sampling function
            subsamples=subsamples.map(_sample_channels_bkt)
        
        # final day of year trig encoding before passing on to static so that model is temporally aware
        def _doy_encoding(feature):
            ts=feature.get('ts')
            year=ee.Number(ee.Date(ts).get('year'))
            doy=ee.Date(ts).difference(ee.Date.fromYMD(year,1,1),'day').add(1)
            angle=doy.divide(365.25).multiply(2*math.pi)
            return feature.set({'doy_sin':angle.sin(),'doy_cos':angle.cos()})
        subsamples=subsamples.map(_doy_encoding)

        # FINALLY return subsamples
        return subsamples

    def phi_static(subsamples:ee.FeatureCollection=subsamples,tileScale:int=2):

        # terrain slope in degrees
        terrain_slope=ee.Terrain.slope(nasa_dem).rename('slope_deg')

        # terrain aspect sin & cos
        aspect_deg=ee.Terrain.aspect(nasa_dem)

        aspect_sin=aspect_deg.multiply(math.pi/180).sin().rename('aspect_sin')
        aspect_cos=aspect_deg.multiply(math.pi/180).cos().rename('aspect_cos')

        # terrain ruggedness
        kernel=ee.Kernel.square(radius=1,units='pixels',normalize=False) # kernel for ruggedness
        tri=nasa_dem.neighborhoodToBands(kernel).subtract(nasa_dem).pow(2).reduce(ee.Reducer.sum()).sqrt().rename('TRI') # tri (terrain ruggedness index)

        # topographic wedness index
        proj=upa.projection()
        sca=upa.multiply(1e6).divide(ee.Image.pixelArea().reproject(proj).sqrt()).max(1) # specific catchment area
        tan_slope=ee.Terrain.slope(nasa_dem.reproject(proj)).multiply(math.pi/180).tan().max(1e-6) # tangent of slope
        twi=sca.divide(tan_slope).log().rename('TWI') # twi (topographic wetness index) derived from sca (specific catchment area) & tan of slope in rad 

        # laplacian curvature (signed second derivative of elevation) derived via built-in laplacian kernel
        proj_lap=ee.Projection(CRS_METRIC).atScale(SCALE_METRIC) # laplacian projection
        curvature=nasa_dem.reproject(proj_lap).convolve(ee.Kernel.laplacian4()).divide(ee.Image.pixelArea().reproject(proj_lap)).rename('curvature_laplace')

        # sand and clay content (4 bands each)
        sand_pc=ee.Image.cat([openlandmap_sand.select(depth).rename(f'sand_content_{depth}') for depth in ['b0','b10','b100','b200']]) # sand content in soil at the shallowest 2 and deepest 2 depths (0cm, 10cm, 100cm, 200cm)
        clay_pc=ee.Image.cat([openlandmap_clay.select(depth).rename(f'clay_content_{depth}') for depth in ['b0','b10','b100','b200']]) # clay content in soil at the shallowest 2 and deepest 2 depths (0cm, 10cm, 100cm, 200cm)
        
        # static volumetric soil water content at 0-5cm and 5-15cm depth at 10kPa, 33kPa, & 1500kPa suctions (6 bands)
        vwc=ee.Image.cat([ee.Image(f"ISRIC/SoilGrids250m/v2_0/wv{suction}").select(f'val_{depth}_Q0_5').resample('bilinear').rename(f'vwc_kPa{suction}_{depth}') for suction in ['0010','0033','1500'] for depth in ['0_5cm','5_15cm']])
        
        # derive z score of ndvi over antecedent 96d window - technically variable over time but can still be treated as static because of >1d cadence
        def ndvi_anom(timestamp:float,days:int):
            # get 16-day interval sequence of ndvi data from modis
            sequence=nasa_modis_terraveg.filterDate(int(timestamp-days*86400000),int(timestamp)).select('NDVI')
            # calculate and return z score based on mean & stdev of ndvi
            return sequence.sort('system:time_start',False).first().subtract(sequence.mean()).divide(sequence.reduce(ee.Reducer.stdDev()).max(1e-6)).resample('bilinear').rename(f'ndvi_anom{days}d')

        # longitude-latitude trig encoding
        def coord_encoding():
            # return an encoding for coordinates so the model can be somewhat spatially conscious
            lonlat=ee.Image.pixelLonLat()
            lat_rad=lonlat.select('latitude').multiply(math.pi/180)
            lon_rad=lonlat.select('longitude').multiply(math.pi/180)
            return ee.Image.cat(lat_rad.sin().rename('lat_sin'),lat_rad.cos().rename('lat_cos'),lon_rad.sin().rename('lon_sin'),lon_rad.cos().rename('lon_cos'))
        lonlat_trig=coord_encoding() # sin and cos of longitude and latitute each for spatial encoding (4 bands)
        
        # stack all static bands and sample (also added wc which doesn't require any processing)
        bands=[terrain_slope,aspect_sin,aspect_cos,tri,twi,curvature,sand_pc,clay_pc,esa_worldcover,vwc,ndvi_anom(time_start,config['features']['windows']['ndvi_anom_d']),lonlat_trig]
        subsamples=ee.Image.cat(bands).unmask(-99).sampleRegions(collection=subsamples,tileScale=tileScale) # unmasked so that non-existent values are just set to -99
        
        # almost done
        return subsamples
    # save fc, sort properties for consistency, drop certain helper properties, return KxD matrix where D is the number of features
    fc=phi_static(phi_dynamic()).getInfo()
    to_drop={'date','ts','u'}

    def get_feature_vector(idx:int): 
        # feature vector generator that extracts features predictably on the clientside from the fc dict
        all_properties=list(fc['features'][idx]['properties'].keys())
        return np.array([fc['features'][idx]['properties'][key] for key in sorted([prop for prop in all_properties if prop not in to_drop])])

    # actually done; return sorted & filtered fc as a matrix
    return np.vstack([get_feature_vector(int(idx)) for idx in range(len(fc['features']))])

In [29]:
testfc=phi(testpointer,10,40,testtimestamp,testtimestamp+24*3600000,24*3600000)

## Background Sampling

In [30]:
# thanks for your time